# Simplifying Definitions with simplify-defun

*Adapted from: A Versatile, Sound Tool for Simplifying Definitions*

*Authors: Alessandro Coglio, Matt Kaufmann, Eric W. Smith*

*ACL2 Workshop 2017*

abstract

  We present a tool, `simplify-defun`, that transforms the
  definition of a given function into a simplified definition of a new
  function, providing a proof checked by ACL2 that the old and new functions are equivalent.
  When appropriate it also generates termination and
  guard proofs for the new function.  We explain how the tool is
  engineered so that these proofs will succeed.  Examples illustrate
  its utility, in particular for program transformation
  in synthesis and verification.

# Introduction

We present a tool, `simplify-defun`, that transforms the
definition of a given function into a simplified definition of a new
function, providing a proof that the old and new functions are equivalent.
When appropriate it also generates termination and
guard proofs for the new function.  Since the generated proofs are submitted to
ACL2, `simplify-defun` need not be trusted: its soundness only depends on
the soundness of ACL2. The new function is a `simplified' version of the
original function in much the same sense that ACL2 `simplifies' terms during
proofs --- via rewrite, type-set, forward chaining, and linear arithmetic rules.

`Simplify-defun` is one of the transformations of APT (Automated Program
Transformations) [citation], an ACL2 library of tools to transform programs
and program specifications with a high degree of automation. APT can be used in
program synthesis, to derive provably correct implementations from formal
specifications via sequences of refinement steps carried out via
transformations. APT can also be used in program analysis, to help verify
existing programs, suitably embedded in the ACL2 logic, by raising their level
of abstraction via transformations that are inverses of the ones used in
stepwise program refinement.  In the APT ecosystem, `simplify-defun` is
useful for simplifying and optimizing definitions generated by other APT transformations (e.g.,
transformations that change data representation), as well as to carry out
rewriting transformations via specific sets of rules (e.g., turning unbounded
integer operations into bounded integer operations and vice versa, under
suitable conditions).  It can also be used to chain together and
propagate sequences of transformations (e.g., rewriting a caller by
replacing a callee with a new version, which may in turn be replaced
with an even newer version, and so on).

The idea of using simplification rules to transform programs
is not new [citation].
The contribution of the work described in this paper
is the realization of that idea in ACL2,
which involves techniques that are specific to this prover and environment
and that leverage its existing capabilities and libraries.
This paper could be viewed as a follow-up to an earlier paper on a
related tool [citation].  But that paper focused
largely on usage, while here, we additionally focus both on
interesting applications and on implementation. (*Note: We thank the
  reviewers of the previous paper for suggesting further discussion of
  implementation.  In a sense, we are finally getting around to that!*)
Moreover, the new tool was implemented from scratch and improves on
the old tool in several important ways, including the following:

- The new tool has significantly more options.  Of special note is
  support for patterns that specify which subterms of the
  definition's body to simplify.

- The new tool has been subjected to a significantly larger
  variety of uses (approximately 300 uses as of mid-January 2017, not
  including artificial tests), illustrating its robustness and
  flexibility.

- The old tool generated events that were written to a file; calls
  of the new tool are event forms that can be placed in a book.  (The
  old tool pre-dated
  [-event](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MAKE-EVENT),
  which is used in the new tool.)

- The new tool takes advantage of the *expander* in community
  book `misc/expander.lisp`, rather than ``rolling its own''
  simplification.  Several recent improvements
  have been made to that book in the course of developing `simplify-defun`, which can benefit other users of the expander.

`Simplify-defun` may be applied to function symbols that have been
defined using `defun`, possibly with mutual recursion --- perhaps indirectly using a macro, for
example,
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DEFUND)
or
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DEFINE).
An analogous tool, `simplify-defun-sk`, may be applied to function
symbols defined with
[-sk](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DEFUN-SK),
but we do not discuss it here.

Notice the underlining above.  Throughout this paper, we underline hyperlinks to topics in the online
[documentation](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html) [citation]
for ACL2 and its books.  We use ACL2 notation freely, abbreviating
with an ellipsis (`...`) to indicate omitted text and sometimes
modifying whitespace in displayed output.

The rest of this section introduces `simplify-defun` via some very simple
examples that illustrate the essence of the tool.
Section [ref] presents some examples of
the tool's use in program transformation; that section provides motivation for
some of the features supported by `simplify-defun`.
Section [ref] summarizes options for controlling this tool.  In
Section [ref] we discuss how `simplify-defun`
circumvents ACL2's mercurial heuristics and sensitivity to
[theories](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_THEORIES)
so that proofs succeed reliably and automatically.  We conclude in
Section [ref].

## Simple Illustrative Examples

We start with a very simple example that captures much of the essence
of `simplify-defun`.

N.B. Run this first in a terminal:
```bash
cert.pl $ACL2_SYSTEM_BOOKS/kestrel/apt/top
```

In [1]:
(include-book "kestrel/apt/simplify" :dir :system)


Summary
Form:  ( INCLUDE-BOOK "kestrel/apt/simplify" ...)
Rules: NIL
Time:  0.58 seconds (prove: 0.00, print: 0.00, other: 0.58)
 "/home/acl2/books/kestrel/apt/simplify.lisp"


In [2]:
(defun f (x)
  (if (zp x) 0 (+ 1 1 (f (+ -1 x)))))


The admission of F is trivial, using the relation O< (which is known
to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT X).  We observe that the type of F is described by the
theorem (OR (AND (INTEGERP (F X)) (< 1 (F X))) (EQUAL (F X) 0)).  We
used primitive type reasoning.

Summary
Form:  ( DEFUN F ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 F


Next, we run `simplify-defun` to produce a new
definition and a `defthm` with the formula shown below.  **Note**: All `defun` forms generated by `simplify-defun`
contain
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DECLARE)
forms, which are generally omitted in this paper; also, whitespace is
liberally edited.

In [3]:
(simplify-defun f)

(DEFUN F$1 (X)
  (DECLARE (XARGS :GUARD T
                  :MEASURE (ACL2-COUNT X)
                  :VERIFY-GUARDS NIL))
  (IF (ZP X) 0 (+ 2 (F$1 (+ -1 X)))))
(DEFTHM F-BECOMES-F$1 (EQUAL (F X) (F$1 X)))


In [4]:
:pf f-becomes-f$1

(EQUAL (F X) (F$1 X))


Several key aspects of a successful `simplify-defun` run are
illustrated above:

- A new function symbol is defined,
using the [numbered-names](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_NUMBERED-NAMES)
utilities.

- The body of the new definition is a simplified version of the
  body of the original definition, but with the old function replaced
  by the new in recursive calls.

- A *'becomes'* theorem is proved, which states the equivalence of
  the old and new function.

This behavior of `simplify-defun` extends naturally to mutual
recursion, in which case a new
[-recursion](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MUTUAL-RECURSION) event is generated together with 'becomes'
theorems.  Consider this definition.

In [5]:
(mutual-recursion
 (defun f1 (x) (if (consp x) (not (f2 (nth 0 x))) t))
 (defun f2 (x) (if (consp x) (f1 (nth 0 x)) t)))


For the admission of F1 and F2 we will use the relation O< (which is
known to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT X) for F1 and (ACL2-COUNT X) for F2.  The non-trivial part
of the measure conjecture is

Goal
(IMPLIES (CONSP X)
         (O< (ACL2-COUNT (NTH 0 X))
             (ACL2-COUNT X))).
Goal'

Q.E.D.

That completes the proof of the measure theorem for F1 and F2.  Thus,
we admit these functions under the principle of definition.  We observe
that the type of F1 is described by the theorem 
(OR (EQUAL (F1 X) T) (EQUAL (F1 X) NIL)) and the type of F2 is described
by the theorem (OR (EQUAL (F2 X) T) (EQUAL (F2 X) NIL)).

Summary
Form:  ( MUTUAL-RECURSION ( DEFUN F1 ...) ...)
Rules: ((:DEFINITION NOT)
        (:DEFINITION NTH)
        (:DEFINITION O-FINP)
        (:DEFINITION O<)
        (:EXECUTABLE-COUNTERPART ZP)
        (:REWRITE ACL2-COUNT-WHEN-MEMBER)
        (:REWRITE SUBSETP-CAR-MEMBER)
        (:REWRITE SUBSETP-REFL)
        (:TYPE-PR

The result presents no surprises when compared to our first example.

In [6]:
(simplify-defun f1)

(MUTUAL-RECURSION
  (DEFUN F1$1 (X)
    (DECLARE (XARGS :GUARD T
                    :MEASURE (ACL2-COUNT X)
                    :VERIFY-GUARDS NIL))
    (IF (CONSP X) (NOT (F2$1 (CAR X))) T))
  (DEFUN F2$1 (X)
    (DECLARE (XARGS :GUARD T
                    :MEASURE (ACL2-COUNT X)
                    :VERIFY-GUARDS NIL))
    (IF (CONSP X) (F1$1 (CAR X)) T)))
(DEFTHM F1-BECOMES-F1$1 (EQUAL (F1 X) (F1$1 X)))
(DEFTHM F2-BECOMES-F2$1 (EQUAL (F2 X) (F2$1 X)))


In [7]:
:pf f1-becomes-f1$1

(EQUAL (F1 X) (F1$1 X))


In [8]:
:pf f2-becomes-f2$1

(EQUAL (F2 X) (F2$1 X))


`Simplify-defun` makes some attempt to preserve structure from the
original definitions.  For example, the body of the definition of `f1`
(above) is stored, as usual, as a translated
[term](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_TERM).
As with most utilities that manipulate ACL2 terms, `simplify-defun` operates on translated terms.  So
the new `defun` event form could easily use the transformed body, shown here; notice that `T` is quoted.

In [9]:
(body 'f1$1 nil (w state))

(IF (CONSP X) (NOT (F2$1 (CAR X))) 'T)


If we naively produced a user-level
([untranslate](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_UNTRANSLATE)d)
term from that body, the result would look quite different from the
original definition's body.

In [10]:
(untranslate (body 'f1$1 nil (w state)) nil (w state))

(OR (NOT (CONSP X))
    (NOT (F2$1 (CAR X))))


Therefore, `simplify-defun` uses the
[-untranslate](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DIRECTED-UNTRANSLATE)
utility to untranslate the new (translated) body, heuristically using the
old body (translated and untranslated) as a guide.
This utility was implemented in support of `simplify-defun`,
but it is of more general use (e.g., in other APT transformations).

There are many ways to control `simplify-defun` by using
keyword arguments, as described in the next two sections.  Here we
show how to limit simplification to specified subterms.  Consider:

In [11]:
(defun g (x y)
  (list (+ (car (cons x y)) 3)
        (* (car (cons y y)) 4)
        (* (car (cons x y)) 5)))


Since G is non-recursive, its admission is trivial.  We observe that
the type of G is described by the theorem 
(AND (CONSP (G X Y)) (TRUE-LISTP (G X Y))).  We used primitive type
reasoning.

Summary
Form:  ( DEFUN G ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 G


The `:simplify-body` keyword option below specifies
simplification of any occurrence of `(car (cons x y))` that is the first
argument of a call to `*`.  The wrapper `:@` indicates the
simplification site, and the underscore (`_`) matches anything.
Notice that, in the result, only the indicated call is simplified.

In [12]:
(simplify-defun g :simplify-body (* (:@ (car (cons x y))) _))

(DEFUN G$1 (X Y)
  (DECLARE (XARGS :GUARD T :VERIFY-GUARDS NIL))
  (LIST (+ (CAR (CONS X Y)) 3)
        (* (CAR (CONS Y Y)) 4)
        (* X 5)))
(DEFTHM G-BECOMES-G$1 (EQUAL (G X Y) (G$1 X Y)))


# Some Applications

This section presents some practical examples of use of
`simplify-defun` in program transformation.  They use some keyword
options, which are described in Section [ref] but we hope
are self-explanatory here.  Not shown here are hints generated by
`simplify-defun` to automate proofs, for example by reusing
previous guards, measures, and guard and termination theorems; this is
covered briefly in subsequent sections.

## Combining a Filter with a Doubly-Recursive Producer

This example shows how `simplify-defun` is used to apply rewrite rules,
to improve the results of other transformations, and to chain
together previous transformation steps.  The main function `f`
below produces all pairs of items from `x` and `y` and then
filters the result to keep only the ``good'' pairs.

In [13]:
(defun pair-with-all (item lst) ;; pair ITEM with all elements of LST
  (if (endp lst)
      nil
    (cons (cons item (car lst))
          (pair-with-all item (cdr lst)))))

(defun all-pairs (x y) ;; make all pairs of items from X and Y
  (if (endp x)
      nil
    (append (pair-with-all (car x) y)
            (all-pairs (cdr x) y))))

(defstub good-pair-p (pair) t) ;; just a place holder

(defun keep-good-pairs (pairs)
  (if (endp pairs)
      nil
    (if (good-pair-p (car pairs))
        (cons (car pairs) (keep-good-pairs (cdr pairs)))
      (keep-good-pairs (cdr pairs)))))

(defun f (x y) (keep-good-pairs (all-pairs x y)))


The admission of PAIR-WITH-ALL is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT LST).  We observe that the type of PAIR-WITH-ALL
is described by the theorem (TRUE-LISTP (PAIR-WITH-ALL ITEM LST)).
We used primitive type reasoning.

Summary
Form:  ( DEFUN PAIR-WITH-ALL ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PAIR-WITH-ALL

The admission of ALL-PAIRS is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT X).  We observe that the type of ALL-PAIRS is described
by the theorem (TRUE-LISTP (ALL-PAIRS X Y)).  We used the :type-prescription
rules BINARY-APPEND and TRUE-LISTP-APPEND.

Summary
Form:  ( DEFUN ALL-PAIRS ...)
Rules: ((:TYPE-PRESCRIPTION BINARY-APPEND)
        (:TYPE-PRESCRIPTION TRUE-LISTP-APPEND))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 ALL-PA

We wish to make `f` more efficient; it should refrain from ever
adding non-good pairs to the result, rather than filtering them out
later.  `F`'s body is `(keep-good-pairs (all-pairs x y))`,
which we can improve by ``pushing'' `keep-good-pairs` into the
`if`-branches of `all-pairs`, using APT's `wrap-output`
transformation (not described here). `Wrap-output` produces a
function and a theorem.

In [14]:
(DEFUN ALL-GOOD-PAIRS (X Y) ; generated by wrap-output
  (IF (ENDP X)
      (KEEP-GOOD-PAIRS NIL)
      (KEEP-GOOD-PAIRS (APPEND (PAIR-WITH-ALL (CAR X) Y)
                               (ALL-PAIRS (CDR X) Y)))))

(DEFTHM RULE1 ; generated by wrap-output
  (EQUAL (KEEP-GOOD-PAIRS (ALL-PAIRS X Y))
         (ALL-GOOD-PAIRS X Y)))


Since ALL-GOOD-PAIRS is non-recursive, its admission is trivial.  We
observe that the type of ALL-GOOD-PAIRS is described by the theorem
(TRUE-LISTP (ALL-GOOD-PAIRS X Y)).  We used the :type-prescription
rule KEEP-GOOD-PAIRS.

Summary
Form:  ( DEFUN ALL-GOOD-PAIRS ...)
Rules: ((:TYPE-PRESCRIPTION KEEP-GOOD-PAIRS))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 ALL-GOOD-PAIRS

Splitter note (see :DOC splitter) for Goal (2 subgoals).
  if-intro: ((:DEFINITION ALL-GOOD-PAIRS))

Subgoal 2
Subgoal 1

Q.E.D.

Summary
Form:  ( DEFTHM RULE1 ...)
Rules: ((:DEFINITION ALL-GOOD-PAIRS)
        (:DEFINITION ALL-PAIRS)
        (:EXECUTABLE-COUNTERPART EQUAL)
        (:EXECUTABLE-COUNTERPART KEEP-GOOD-PAIRS)
        (:FAKE-RUNE-FOR-TYPE-SET NIL))
Splitter rules (see :DOC splitter):
  if-intro: ((:DEFINITION ALL-GOOD-PAIRS))
Time:  0.01 seconds (prove: 0.01, print: 0.00, other: 0.00)
Prover steps counted:  583
 RULE1


Below, we will apply `rule1` to simplify
`f`.  But first we will further transform `all-good-pairs`. It
can be simplified in three ways.  First, `(keep-good-pairs nil)`
can be evaluated.  Second, we can push the call to `keep-good-pairs` over the `append` using this rule.

In [15]:
(defthm keep-good-pairs-of-append
  (equal (keep-good-pairs (append x y))
         (append (keep-good-pairs x) (keep-good-pairs y))))


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Three induction schemes are
suggested by this conjecture.  These merge into two derived induction
schemes.  However, one of these is flawed and so we are left with one
viable candidate.  

We will induct according to a scheme suggested by (KEEP-GOOD-PAIRS X),
while accommodating (APPEND X Y).

These suggestions were produced using the :induction rules BINARY-APPEND
and KEEP-GOOD-PAIRS.  If we let (:P X Y) denote *1 above then the induction
scheme we'll use is
(AND (IMPLIES (AND (NOT (ENDP X))
                   (NOT (GOOD-PAIR-P (CAR X)))
                   (:P (CDR X) Y))
              (:P X Y))
     (IMPLIES (AND (NOT (ENDP X))
                   (GOOD-PAIR-P (CAR X))
                   (:P (CDR X) Y))
              (:P X Y))
     (IMPLIES (ENDP X) (:P X Y))).
This induction is justified by the same argument used to admit 
KEEP-GOOD-PAIRS.  When applied to the goal at ha

Third, note that `all-good-pairs`, despite being a transformed
version of `all-pairs`, is not recursive (it calls the old
function `all-pairs`), but we want it to be recursive.
After `keep-good-pairs` is pushed over the `append`, it will be composed with the call of `all-pairs`, which
is the exact pattern that `rule1` can rewrite to a call to `all-good-pairs`.  `Simplify-defun` applies these
simplifications.

In [16]:
(simplify-defun all-good-pairs)

(DEFUN ALL-GOOD-PAIRS$1 (X Y)
  (DECLARE (XARGS :GUARD T :VERIFY-GUARDS NIL))
  (IF (ENDP X)
      NIL
    (APPEND (KEEP-GOOD-PAIRS (PAIR-WITH-ALL (CAR X) Y))
            (ALL-GOOD-PAIRS$1 (CDR X) Y))))
(DEFTHM ALL-GOOD-PAIRS-BECOMES-ALL-GOOD-PAIRS$1
  (EQUAL (ALL-GOOD-PAIRS X Y)
         (ALL-GOOD-PAIRS$1 X Y)))


Note that the new function is recursive.  This is because
`rule1` introduced a call to `all-good-pairs`, which
`simplify-defun` then renamed to `all-good-pairs$1` (it always renames
recursive calls). We have made some progress pushing `keep-good-pairs` closer
to where the pairs are created. Now, `all-good-pairs$1` can be further
simplified.  Observe that its body contains composed calls of
`keep-good-pairs` and `pair-with-all`.  We can optimize this term using APT's
`producer-consumer` transformation (not described here) to combine the
creation of the pairs with the filtering of good pairs.  As usual, a new
function and a theorem are produced.

In [17]:
(DEFUN PAIR-WITH-ALL-AND-FILTER (ITEM LST) ; generated by producer-consumer
  (IF (ENDP LST)
      NIL
      (IF (GOOD-PAIR-P (CONS ITEM (CAR LST)))
          (CONS (CONS ITEM (CAR LST))
                (PAIR-WITH-ALL-AND-FILTER ITEM (CDR LST)))
          (PAIR-WITH-ALL-AND-FILTER ITEM (CDR LST)))))

(DEFTHM RULE2 ; generated by producer-consumer
  (EQUAL (KEEP-GOOD-PAIRS (PAIR-WITH-ALL ITEM LST))
         (PAIR-WITH-ALL-AND-FILTER ITEM LST)))


The admission of PAIR-WITH-ALL-AND-FILTER is trivial, using the relation
O< (which is known to be well-founded on the domain recognized by O-P)
and the measure (ACL2-COUNT LST).  We observe that the type of 
PAIR-WITH-ALL-AND-FILTER is described by the theorem 
(TRUE-LISTP (PAIR-WITH-ALL-AND-FILTER ITEM LST)).  We used primitive
type reasoning.

Summary
Form:  ( DEFUN PAIR-WITH-ALL-AND-FILTER ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PAIR-WITH-ALL-AND-FILTER

*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

Perhaps we can prove *1 by induction.  Two induction schemes are suggested
by this conjecture.  Subsumption reduces that number to one.  

We will induct according to a scheme suggested by 
(PAIR-WITH-ALL-AND-FILTER ITEM LST), while accommodating 
(PAIR-WITH-ALL ITEM LST).

These suggestions were produced using the :induction rules PAIR-WITH-ALL
and PAIR-WITH-ALL-AND-FILTER.  If we let (:P ITEM

`Pair-with-all-and-filter` immediately discards non-good pairs, saving work compared to filtering them out later.
Now `simplify-defun` can change `all-good-pairs$1` by
applying `rule2`.

In [18]:
(simplify-defun all-good-pairs$1)

(DEFUN ALL-GOOD-PAIRS$2 (X Y)
  (DECLARE (XARGS :GUARD T
                  :MEASURE (ACL2-COUNT X)
                  :VERIFY-GUARDS NIL))
  (IF (ENDP X)
      NIL
    (APPEND (PAIR-WITH-ALL-AND-FILTER (CAR X) Y)
            (ALL-GOOD-PAIRS$2 (CDR X) Y))))
(DEFTHM ALL-GOOD-PAIRS$1-BECOMES-ALL-GOOD-PAIRS$2
  (EQUAL (ALL-GOOD-PAIRS$1 X Y)
         (ALL-GOOD-PAIRS$2 X Y)))


Finally, we apply `simplify-defun` to transform `f` by applying all of the
preceding rewrites in succession, introducing `all-good-pairs`, which is in
turn replaced with `all-good-pairs$1` and then `all-good-pairs$2`.

In [19]:
(simplify-defun f :new-name f-fast)
:pf f-becomes-f-fast

(DEFUN F-FAST (X)
  (DECLARE (XARGS :GUARD T :VERIFY-GUARDS NIL))
  (IF (ZP X) 0 (+ 2 (F$1 (+ -1 X)))))
(DEFTHM F-BECOMES-F-FAST (EQUAL (F X) (F-FAST X)))
(EQUAL (F X) (F-FAST X))


This builds a fast version of `f` and a theorem proving it equal to `f`.

## Converting between Unbounded and Bounded Integer Operations

Popular programming languages like C and Java
typically use bounded integer types and operations,
while requirements specifications
typically use unbounded integer types and operations.
Thus, synthesizing a C or Java program from a specification,
or proving that a C or Java program complies with a specification,
often involves showing that unbounded and bounded integers are ``equivalent''
under the preconditions stated by the specification.

Consider this Java implementation
of Bresenham's line drawing algorithm [citation]
for the first octant.%
 (*Note: This algorithm computes a best-fit discrete line
  using only integer operations.
  Understanding the algorithm is not necessary
  for the purpose of this `simplify-defun` example.*)

```
// draw a line from (0, 0) to (a, b), where 0 <= b <= a <= 1,000,000:
static void drawLine(int a, int b) {
    int x = 0, y = 0, d = 2 * b - a;
    while (x <= a) {
        drawPoint(x, y); // details unimportant
        x++;
        if (d >= 0) { y++; d += 2 * (b - a); }
        else { d += 2 * b; }
    }
}
```

Assuming the screen width and height are less than 1,000,000 pixels,
none of the two's complement 32-bit integer operations in the Java method
 wrap around. So they
 could be replaced with corresponding unbounded integer operations,
as shown below.
This replacement raises the level of abstraction
and helps verify the functional correctness of the method.

The Java code above can be represented as shown below in ACL2
(see the paper's supporting materials for full details), where:

- `Int32p` recognizes
  some representation of Java's two's complement 32-bit integers,
  whose details are unimportant.

- `Int32` converts an ACL2 integer in $[-2^{31},2^{31})$
  (i.e., an `x` such that `(signed-byte-p 32 x)` holds)
  to the corresponding representation in `int32p`.

- `Int` converts a representation in `int32p`
  to the corresponding ACL2 integer in $[-2^{31},2^{31})$.

- `Add32`, `sub32`, and `mul32`
  represent Java's two's complement 32-bit
  addition, subtraction, and multiplication operations.

- `Lte32` and `gte32`
  represent Java's two's complement 32-bit
  less-than-or-equal-to and greater-than-or-equal-to operations.

- `Drawline-loop` represents the loop as a state transformer,
  whose state consists of
  `a`, `b`, `x`, `y`, `d`, and the screen.
  This function is ``defined'' only where the loop invariant holds.
  The guard verification of this function implies
  the preservation of the loop invariant.

- `Drawline` represents the method as a function that maps
  `a`, `b`, and the current screen to an updated screen.
  This function is ``defined''only where the precondition holds.
  The guard verification of this function implies
  the establishment of the loop invariant.

The Axe tool [citation] can automatically generate
a representation similar to this one from Java (byte)code,
with some additional input from the user (e.g., part of the loop invariant).

In [ ]:
(defun drawpoint (x y screen)
  (declare (xargs :guard (and (int32p x) (int32p y))))
  ...) ; returns updated screen, details unimportant

(defun precond (a b) ; precondition of the method
  (declare (xargs :guard t))
  (and (int32p a) ; Java type of a
       (int32p b) ; Java type of b
       (<= 0 (int b))
       (<= (int b) (int a))
       (<= (int a) 1000000)))

(defun invar (a b x y d) ; loop invariant of the method
  (declare (xargs :guard t))
  (and (precond a b)
       (int32p x) ; Java type of x
       (int32p y) ; Java type of y
       (int32p d) ; Java type of d
       ...)) ; conditions on x, y, and d, details unimportant

(defun drawline-loop (a b x y d screen) ; loop of the method
  (declare (xargs :guard (invar a b x y d)
                  ...)) ; measure and (guard) hints, details unimportant
  (if (invar a b x y d)
      (if (not (lte32 x a))
          screen ; exit loop
        (drawline-loop a b
                       (add32 x (int32 1))
                       (if (gte32 d (int32 0))
                           (add32 y (int32 1))
                         y)
                       (if (gte32 d (int32 0))
                           (add32 d (mul32 (int32 2) (sub32 b a)))
                         (add32 d (mul32 (int32 2) b)))
                       (drawpoint x y screen)))
    :undefined))

(defun drawline (a b screen) ; method
  (declare (xargs :guard (precond a b)
                  ...)) ; guard hints, details unimportant
  (if (precond a b)
      (drawline-loop a b
                     (int32 0) ; x
                     (int32 0) ; y
                     (sub32 (mul32 (int32 2) b) a) ; d
                     screen)
    :undefined))

The following rewrite rules are disabled
because their right-hand sides are not generally ``simpler'' or ``better''
than their left-hand sides.
But when passed to the `:enable` option of `simplify-defun`,
which instructs the tool to use these rules in the expander,
they systematically replace bounded integer operations
with their unbounded counterparts.

In [ ]:
(defthmd add32-to-+  (equal (add32 x y) (int32 (+ (int x) (int y)))))
(defthmd sub32-to--  (equal (sub32 x y) (int32 (- (int x) (int y)))))
(defthmd mul32-to--  (equal (mul32 x y) (int32 (* (int x) (int y)))))
(defthmd lte32-to-<= (equal (lte32 x y) (<= (int x) (int y))))
(defthmd gte32-to-<= (equal (gte32 x y) (>= (int x) (int y))))

Since these rewrite rules are unconditional,
the replacement always occurs,
but subterms of the form `(int (int32 ...))` are generated.
For instance,
the term `(add32 d (mul32 (int32 2) b))` above becomes
`(int32 (+ (int d) (int (int32 (* (int (int32 2)) (int b))))))`.
These `(int (int32 ...))` terms can be simplified
via the following conditional rewrite rule
(which the expander in `simplify-defun` uses by default,
since it is an enabled rule).
Relieving the hypotheses of this rewrite rule's applicable instances
amounts to showing that each bounded integer operation does not wrap around
in the expressions under consideration.

In [22]:
(defthm int-of-int32
  (implies (signed-byte-p 32 x)
           (equal (int (int32 x)) x)))



ACL2 Error [Translate] in ( DEFTHM INT-OF-INT32 ...):  The symbol INT
(in package "ACL2") has neither a function nor macro definition in
ACL2.  Please define it.  See :DOC near-misses.  Note:  this error
occurred in the context (INT (INT32 X)).


Summary
Form:  ( DEFTHM INT-OF-INT32 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( DEFTHM INT-OF-INT32 ...):  See :DOC failure.

******** FAILED ********


Applying `simplify-defun` to `drawline-loop` and `drawline`
yields the desired results.
In this case, the `int-of-int32` hypotheses are automatically relieved.
These uses of `simplify-defun` show a practical use of the pattern feature:
`invar` and `precond` must be enabled
to relieve the `int-of-int32` hypotheses,
but we want the generated function to keep them unopened;
so we use a pattern that limits the simplification
to the true branches of the `if`s.
The 'becomes' theorem generated by the first `simplify-defun`
is used by the second to have `drawline$1`
call `drawline-loop$1` instead of `drawline-loop`;
this is another example of propagating transformations.

In [38]:
(simplify-defun drawline-loop
                :simplify-body (if _ @ _)
                :enable (add32-to-+ sub32-to-- mul32-to-* lte32-to-<= gte32-to->=))

(simplify-defun drawline
                :simplify-body (if _ @ _)
                :enable (add32-to-+ sub32-to-- mul32-to-* lte32-to-<= gte32-to->=))



ACL2 Error in (APT::SIMPLIFY DRAWLINE-LOOP ...):  The first argument
of the SIMPLIFY transformation must either be a known function symbol
or a term that is neither a variable nor a constant.  The argument
DRAWLINE-LOOP is thus illegal.



ACL2 Error in (APT::SIMPLIFY DRAWLINE ...):  The first argument of
the SIMPLIFY transformation must either be a known function symbol
or a term that is neither a variable nor a constant.  The argument
DRAWLINE is thus illegal.



The resulting expressions have
the `int` conversion at the variable leaves
and the `int32` conversion at the roots.
APT's isomorphic data transformation (not discussed here)
can eliminate them by changing the data representation
of the functions' arguments,
generating functions that no longer deal with bounded integers.
The resulting functions are more easily proved
to satisfy the high-level functional specification of the algorithm,
namely that it produces a best-fit discrete line
(this proof is not discussed here).

The bounded-to-unbounded operation rewriting technique shown here,
followed by the isomorphic data transformation mentioned above,
should have general applicability.
Proving that bounded integer operations do not wrap around
may be arbitrarily hard:
when the `int-of-int32` hypotheses cannot be relieved automatically,
the user may have to prove lemmas to help `simplify-defun`.
When a Java computation is supposed to wrap around
(e.g., when calculating a hash),
the specification must explicitly say that,
and slightly different rewrite rules may be needed.
When synthesizing code (as opposed to analyzing code as in this example),
it should be possible to use a similar technique
with rewrite rules that turn
unbounded integer operations into their bounded counterparts,
``inverses'' of the rules `add32-to-+`, `sub32-to--`, etc.

# Options

This section very briefly summarizes the keyword arguments of `simplify-defun`.  Here we assume that the given function's
  definition is not mutually recursive; for that case and other
  details, see the
  [XDOC](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_XDOC)
  topic for simplify-defun, provided by the supporting materials.

**Assumptions.**  A list of assumptions under which the body is
simplified can be specified by the `:assumptions` keyword.
Or, keyword `:hyp-fn` can specify the assumptions using
a function symbol.

**Controlling the Result.**  By default, the new function symbol is
[`disable`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DISABLE)d
if and only if the input function symbol is disabled.  However, that
default can be overridden with keyword `:function-disabled`.
Similarly, the measure, guard verification, and non-executability
status come from the old function symbol but can be overridden by
keywords `:measure`, `:verify-guards`, and `:non-executable`, respectively.  The 'becomes' theorem is
[`enable`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_ENABLE)d
by default, but this can be overridden by keyword `:theorem-disabled`.  Keywords can also specify the names for the new
function symbol (`:new-name`) and 'becomes' theorem (`:theorem-name`).  The new function body is produced, by default, using the
[`-untranslate`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_DIRECTED-UNTRANSLATE)
utility (see Section [ref]); but keyword `:untranslate` can specify to use the ordinary untranslate operation
or even to leave the new body in translated form.  Finally, by default
the new function produces results equal to the old; however, an
equivalence relation between the old and new results can be specified
with keyword `:equiv`, which is used in the statement of the
'becomes' theorem.

**Specifying Theories.**  The `:theory` keyword specifies the
theory to be used when simplifying the definition; alternatively, `:disable` and `:enable` keywords can be used for this purpose.
Keyword `:expand` can be used with any (or none) of these to
specify terms to expand, as with ordinary `:expand` hints for the
prover.  Similarly, there are keywords `:assumption-theory`, `:assumption-disable`, and `:assumption-enable` for controlling
the theory used when proving that assumptions are preserved by
recursive calls (an issue discussed in Section [ref]).
There are also keywords `:measure-hints` and `:guard-hints`
with the obvious meanings.

**Specifying Simplification.**  By default, `simplify-defun`
attempts to simplify the body of the given function symbol, but not
its guard or measure.  Keywords `:simplify-body`, `:simplify-guard`, and `:simplify-measure` can override that
default behavior.  By default, `simplify-defun` fails if
it attempts to simplify the body but fails to do so, though there is
no such requirement for the guard or measure; keyword `:must-simplify` can override those defaults.

**Output Options.**  The keyword `:show-only` causes `simplify-defun` not to change the
[`world`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/?topic=ACL2____WORLD), but instead to show how it
expands into primitive events (see Section [ref]).
If `:show-only` is `nil` (the default), then by default, the
new definition is printed when `simplify-defun` is successful;
keyword `:print-def` can suppress that printing.  Finally, a `:verbose` option can provide extra output.

# Implementation

`simplify-defun` is designed to apply the *expander*,
specifically, function `tool2-fn` in community book `misc/expander.lisp`, which provides an interface to the
ACL2 rewriter in a context based on the use of forward-chaining
and linear arithmetic.  The goal is to simplify the definition as
specified and to arrange that all resulting proofs succeed fully
automatically and quickly.

This section discusses how `simplify-defun` achieves this goal,
with a few (probably infrequent) exceptions in the case of proofs for
guards, termination, or (discussed below) that assumptions are
preserved by recursive calls.  First, we explain the full form
generated by a call of `simplify-defun`, which we call its *expansion*; this form carefully orchestrates the proofs.  Then we
dive deeper by exploring the proof of the 'becomes' theorem, focusing on
the use of functional instantiation.  Next we see how the expansion is
modified when assumptions are used.  Finally we provide a
few brief implementation notes.

One can experiment using the supporting materials: see the book `simplify-defun-tests.lisp`; or simply include the book `simplify-defun`, define a function `f`, and then evaluate `(simplify-defun f :show-only t)`, perhaps adding options, to see the
expansion.

The implementation takes advantage of many features offered by ACL2
for system building (beyond its prover engine), including for example
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_ENCAPSULATE),
[-event](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MAKE-EVENT),
[-output](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_WITH-OUTPUT).
We say a bit more about this at the end of the section.

## The `Simplify-defun` Expansion

We illustrate the expansion generated by `simplify-defun` using
the following definition, which is the first example of
Section [ref] but with a guard added.

In [12]:
(defun fn (x)
 (declare (xargs :guard (natp x)))
 (if (zp x) 0 (+ 1 1 (f (+ -1 x)))))


Since FN is non-recursive, its admission is trivial.  We observe that
the type of FN is described by the theorem 
(OR (AND (INTEGERP (FN X)) (< 1 (FN X))) (EQUAL (FN X) 0)).  We used
primitive type reasoning and the :type-prescription rule F.

Computing the guard conjecture for FN....

The guard conjecture for FN is trivial to prove, given the :compound-
recognizer rules NATP-COMPOUND-RECOGNIZER and ZP-COMPOUND-RECOGNIZER,
primitive type reasoning and the :type-prescription rule F.  FN is
compliant with Common Lisp.

Summary
Form:  ( DEFUN FN ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER)
        (:COMPOUND-RECOGNIZER ZP-COMPOUND-RECOGNIZER)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION F))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FN


The expansion is an
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_ENCAPSULATE)
event.  We can see the expansion by evaluating the form `(simplify-defun f :show-only t)`, which includes the indicated
four sections to be discussed below.

The expansion structure:

;; convert this to markdown
```latex
(encapsulate nil
 {\em{prelude}}
 {\em{new defun form}}
 (local (progn {\em{local events}}))
 {\em{\becomes theorem}})
```

In [13]:
(simplify-defun fn :show-only t)

(ENCAPSULATE-REPORT-ERRORS
 (APT::SIMPLIFY . FN)
 NIL (LOGIC)
 (SET-INHIBIT-WARNINGS "theory")
 (SET-IGNORE-OK T)
 (SET-IRRELEVANT-FORMALS-OK T)
 (LOCAL (INSTALL-NOT-NORMALIZED FN
                                :DEFTHM-NAME 'FN$NOT-NORMALIZED
                                :ALLP NIL))
 (LOCAL
  (PROGN
   (MAKE-EVENT
     (LET
      ((APT::THY (MERGE-SORT-LEXORDER
                      (UNION-EQUAL '((:DEFINITION SYNP)
                                     (:EXECUTABLE-COUNTERPART BINARY-+)
                                     (:REWRITE FOLD-CONSTS-IN-+))
                                   (CONGRUENCE-THEORY-FN :HERE (W STATE))))))
      (LIST 'DEFCONST
            '*FN-RUNES*
            (LIST 'QUOTE APT::THY))))
   (ON-FAILURE
    (ORELSE
      (WITH-OUTPUT :OFF :ALL
                   (DEFUN FN$1 (X)
                     (DECLARE (XARGS :GUARD (NATP X)
                                     :VERIFY-GUARDS NIL))
                     (IF (ZP X) 0 (+ 2 (F (+ -1 X))))))
      (APT::PREFIX

### Prelude

The prelude is mostly independent of `fn`, but the name of `fn`
is supplied to `install-not-normalized`.

In [14]:
(SET-INHIBIT-WARNINGS "theory")

 ("theory")


In [15]:
(SET-IGNORE-OK T)

 T


In [16]:
(SET-IRRELEVANT-FORMALS-OK T)

 T


In [17]:
(LOCAL (INSTALL-NOT-NORMALIZED FN))


To verify that the four encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(SET-IGNORE-OK T)
T


ACL2 !>>(SET-IRRELEVANT-FORMALS-OK T)
T


ACL2 !>>(DEFTHM FN$NOT-NORMALIZED
          (EQUAL (FN X)
                 (IF (ZP X)
                     '0
                   (BINARY-+ '1
                             (BINARY-+ '1 (F (BINARY-+ '-1 X))))))
          :HINTS (("Goal" :BY FN))
          :RULE-CLASSES ((:DEFINITION :INSTALL-BODY T)))

ACL2 Warning [Non-rec] in ( DEFTHM FN$NOT-NORMALIZED ...):  A :DEFINITION
rule generated from FN$NOT-NORMALIZED will be triggered only by terms
containing the function symbol FN, which has a non-recursive definition.
Unless this definition is disabled, this rule is unlikely ever to be
used.


Q.E.D.

Summary
Form:  ( DEFTHM FN$NOT-NORMALIZED ...)
Rules: NIL
Hint-events: ((:BY FN))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, 

In [18]:
(LOCAL (SET-DEFAULT-HINTS NIL))

 NIL


In [19]:
(LOCAL (SET-OVERRIDE-HINTS NIL))


The event ( TABLE DEFAULT-HINTS-TABLE ...) is redundant.  See :DOC
redundant-events.
 NIL


The first form above avoids warnings due to the use of small theories for
directing the prover.  The next two forms allow simplification to make
some formals unused or irrelevant in the new definition.  The use of
[`-not-normalized`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_INSTALL-NOT-NORMALIZED)
is a bit subtle perhaps, but not complicated: by default, ACL2 stores
a simplified, or
{[`normalize`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_NORMALIZE)d},
body for a function symbol, but `simplify-defun` is intended to generate
a definition based on the body $b$ of the old definition as it was submitted, not on
the normalization of $b$.  Using `install-not-normalized`
arranges that `:expand` hints for `fn` will use the
unnormalized body.  This supports the proofs generated by `simplify-defun`, by supporting reasoning about the unnormalized
bodies of both the old and new functions.
Finally, the last two forms
guarantee that the global environment will not sabotage the proof.

Space does not permit discussion of the handling of
[`-recursion`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MUTUAL-RECURSION).
We mention here only that the form `(SET-BOGUS-MUTUAL-RECURSION-OK T)` is then added to the prelude, in case some of the (mutual)
recursion disappears with simplification.

### New Defun Form

The following new definition has the same simplified body as for the
corresponding example in Section [ref].  As before, the new
body is produced by running the expander on the unnormalized body of
`f`.

In [20]:
(DEFUN FN$1 (X)
  (DECLARE (XARGS :NORMALIZE NIL
                  :GUARD (NATP X)
                  :MEASURE (ACL2-COUNT X)
                  :VERIFY-GUARDS T
                  :GUARD-HINTS (("Goal" :USE (:GUARD-THEOREM FN)) ...)
                  :HINTS (("Goal" :USE (:TERMINATION-THEOREM FN)) ...)))
  (IF (ZP X) 0 (+ 2 (FN$1 (+ -1 X)))))


***********************************************
************ ABORTING from raw Lisp ***********
********** (see :DOC raw-lisp-error) **********
Error:  too many dots

          Stream: #<SYNONYM-STREAM :SYMBOL SB-SYS:*STDIN* {10000BBCC3}>
***********************************************

The message above might explain the error.  If not, and
if you didn't cause an explicit interrupt (Control-C),
then it may help to see :DOC raw-lisp-error.

To enable breaks into the debugger (also see :DOC acl2-customization):
(SET-DEBUGGER-ENABLE T)


The
[guard](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_GUARD)
and
[measure](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MEASURE)
are (by default) inherited from `fn`.  Since `fn` is
guard-verified, then by default, so is the new function.
The uses of
[:{GUARD-THEOREM](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_GUARD-THEOREM)}
and
[:{TERMINATION-THEOREM](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_TERMINATION-THEOREM)}
are designed to make the guard and termination proofs automatic and
fast in most cases, and were implemented in support of the APT project.

### Local Events

The local events are how `simplify-defun` carefully arranges for
proofs to succeed automatically and fast, in most cases.  The first
local event defines a
[theory](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_THEORY)
to be used when proving equality of the body of `f` with a
simplified version.  It is the union of the
[runes](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_RUNES)
reported by the expander when simplifying the body, with the set of
all [`:congruence`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_CONGRUENCE)
and [`:equivalence`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_EQUIVALENCE)
runes, since these are not tracked by the ACL2 rewriter. (*Note: We
  originally generated a form `(defconst *f-runes* ...)` that
  listed all runes from the `union-equal` call; but the congruence
  theory made that list long and distracting when viewing the
  expansion with `:show-only t`.  Note that the congruence theory
  includes `:equivalence` runes, which after all represent
  congruence rules for diving into calls of equivalence relations.*)

In [21]:
(MAKE-EVENT (LET ((THY (UNION-EQUAL '((:REWRITE FOLD-CONSTS-IN-+)
                                      (:EXECUTABLE-COUNTERPART BINARY-+)
                                      (:DEFINITION SYNP))
                                    (CONGRUENCE-THEORY (W STATE)))))
                 (LIST 'DEFCONST '*F-RUNES* (LIST 'QUOTE THY))))



ACL2 Error in ( MAKE-EVENT (LET ...)):  Global variables, such as WORLD,
are not allowed.  See :DOC ASSIGN and :DOC @.


Summary
Form:  ( MAKE-EVENT (LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( MAKE-EVENT (LET ...)):  See :DOC failure.

******** FAILED ********


The second local event proves the equality of the body with
its simplified version.  Notice that the latter is still in terms of
`f`; the new function symbol, `f$1`, will be introduced
later.  The
[proof-builder](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_PROOF-BUILDER)
`:instructions` are carefully generated to guarantee that the
proof succeeds; in this simple example, they first put the
proof-builder in the smallest theory that should suffice (for
efficiency), then simplify the old body (the first argument of the
`equal` call), and then prove the resulting equality (which at
that point should be the equality of two identical terms).

In [29]:
(DEFTHM F-BEFORE-VS-AFTER-0
  (EQUAL (IF (ZP X) 0 (+ 1 1 (F (+ -1 X))))
         (IF (ZP X) 0 (+ 2 (F (+ -1 X)))))
  :INSTRUCTIONS ...
  :RULE-CLASSES NIL)


***********************************************
************ ABORTING from raw Lisp ***********
********** (see :DOC raw-lisp-error) **********
Error:  too many dots

          Stream: #<SYNONYM-STREAM :SYMBOL SB-SYS:*STDIN* {10000BBCC3}>
***********************************************

The message above might explain the error.  If not, and
if you didn't cause an explicit interrupt (Control-C),
then it may help to see :DOC raw-lisp-error.

To enable breaks into the debugger (also see :DOC acl2-customization):
(SET-DEBUGGER-ENABLE T)


The third local event is as follows.

In [22]:
(COPY-DEF FN$1
          :HYPS-FN NIL
          :HYPS-PRESERVED-THM-NAMES NIL
          :EQUIV EQUAL)



HARD ACL2 ERROR in FN-IS-FN-COPY:  Unexpected error!  (Sorry about
the obscure error message; please send it to Matt, who will investigate.)



ACL2 Error [Evaluation] in ( MAKE-EVENT (COPY-DEF-EVENT ...)):  Evaluation
aborted.  To debug see :DOC print-gv, see :DOC trace, and see :DOC
wet.


Summary
Form:  ( MAKE-EVENT (COPY-DEF-EVENT ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( MAKE-EVENT (COPY-DEF-EVENT ...)):  See :DOC
failure.

******** FAILED ********


This macro call introduces a constrained function symbol,
`f$1-copy`, whose constraint results from the definitional axiom
for `f$1` by replacing `f$1` with `f$1-copy`.  It
also proves the two functions equivalent using a trivial induction in
a tiny theory, to make the proof reliable and fast.

In [30]:
The generated theorems look like:

```lisp
(DEFTHM F$1-COPY-DEF
  (EQUAL (F$1-COPY X)
         (IF (ZP X)
             '0
             (BINARY-+ '2 (F$1-COPY (BINARY-+ '-1 X)))))
  :HINTS ...
  :RULE-CLASSES ((:DEFINITION :INSTALL-BODY T
                              :CLIQUE (F$1-COPY)
                              :CONTROLLER-ALIST ((F$1-COPY T)))))

(LOCAL (IN-THEORY '((:INDUCTION F$1) F$1-COPY-DEF (:DEFINITION F$1))))

(DEFTHM F$1-IS-F$1-COPY
  (EQUAL (F$1 X) (F$1-COPY X))
  :HINTS (("Goal" :INDUCT (F$1 X)))
  :RULE-CLASSES NIL)
```



ACL2 Error in TOP-LEVEL:  Global variables, such as THE, are not allowed.
See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as GENERATED, are
not allowed.  See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as THEOREMS, are not
allowed.  See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as LOOK, are not allowed.
See :DOC ASSIGN and :DOC @.


***********************************************
************ ABORTING from raw Lisp ***********
********** (see :DOC raw-lisp-error) **********
Error:  illegal terminating character after a colon: #\ 
          
          Stream: #<SYNONYM-STREAM :SYMBOL SB-SYS:*STDIN* {10000BBCC3}>
***********************************************

The message above might explain the error.  If not, and
if you didn't cause an explicit interrupt (Control-C),
then it may help to see :DOC raw-lisp-error.

To enable breaks into the debugger (also see :DOC acl2-customization):
(SET

The last local event is the lemma for proving the 'becomes'
theorem.  In Section [ref] below we explore this use of
functional instantiation.

In [31]:
The lemma for proving the 'becomes' theorem:

```lisp
(DEFTHM F-BECOMES-F$1-LEMMA
  (EQUAL (F$1 X) (F X))
  :HINTS (("Goal"
           :BY (:FUNCTIONAL-INSTANCE F$1-IS-F$1-COPY (F$1-COPY F))
           :IN-THEORY (UNION-THEORIES (CONGRUENCE-THEORY WORLD)
                                      (THEORY 'MINIMAL-THEORY)))
          '(:USE (F-BEFORE-VS-AFTER-0 F$NOT-NORMALIZED))))
```



ACL2 Error in TOP-LEVEL:  Global variables, such as THE, are not allowed.
See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as LEMMA, are not
allowed.  See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as FOR, are not allowed.
See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as PROVING, are not
allowed.  See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as THE, are not allowed.
See :DOC ASSIGN and :DOC @.

BECOMES

***********************************************
************ ABORTING from raw Lisp ***********
********** (see :DOC raw-lisp-error) **********
Error:  illegal terminating character after a colon: #\ 
          
          Stream: #<SYNONYM-STREAM :SYMBOL SB-SYS:*STDIN* {10000BBCC3}>
***********************************************

The message above might explain the error.  If not, and
if you didn't cause an explicit interrupt (Control-C),
then it may he

### 'Becomes' Theorem

The 'becomes' theorem in this example states the same theorem as
its lemma above (though we will see in
Section [ref] that this is not always be the case).
The `:in-theory` hint serves to keep
the ACL2 rewriter from bogging down during the proof.

In [32]:
(DEFTHM F-BECOMES-F$1
  (EQUAL (F X) (F$1 X))
  :HINTS (("Goal" :USE F-BECOMES-F$1-LEMMA :IN-THEORY NIL)))

''LISP

The event ( DEFTHM F-BECOMES-F$1 ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFTHM F-BECOMES-F$1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


## Proving the 'Becomes' Theorem

Next we see how functional instantiation is used in proving the
'becomes' theorem above, or more precisely, its local lemma.  Recall that
above, a
[`:by`](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_BY)
hint is used that replaces `f$1-copy` by `f` in the lemma
`f$1-is-f$1-copy`, `(EQUAL (F$1 X) (F$1-COPY X))`, to
prove: `(EQUAL (F X) (F$1 X))`.  That substitution works
perfectly (modulo commuting the equality, which the `:by` hint
tolerates), but it requires proving the following property, which
states that `f` satisfies the constraint for `f$1-copy`.

In [33]:
The property that `f` must satisfy:

```lisp
(EQUAL (f X)
       (IF (ZP X)
           '0
           (BINARY-+ '2 (f (BINARY-+ '-1 X)))))
```

''THE


ACL2 Error in TOP-LEVEL:  Global variables, such as PROPERTY, are not
allowed.  See :DOC ASSIGN and :DOC @.



ACL2 Error in TOP-LEVEL:  Global variables, such as THAT, are not allowed.
See :DOC ASSIGN and :DOC @.

F
MUST

***********************************************
************ ABORTING from raw Lisp ***********
********** (see :DOC raw-lisp-error) **********
Error:  illegal terminating character after a colon: #\ 
          
          Stream: #<SYNONYM-STREAM :SYMBOL SB-SYS:*STDIN* {10000BBCC3}>
***********************************************

The message above might explain the error.  If not, and
if you didn't cause an explicit interrupt (Control-C),
then it may help to see :DOC raw-lisp-error.

To enable breaks into the debugger (also see :DOC acl2-customization):
(SET-DEBUGGER-ENABLE T)


But the right-hand side is exactly what was produced by
applying the expander to the body of `f`.  If we look at the `:hints` in `f-becomes-f$1-lemma` above, we see that after
using functional instantiation (with the `:by` hint), a computed
hint completes the proof by using two facts: `f-before-vs-after-0`
(the second local lemma above), which equates the two bodies; and
`f$not-normalized`, which equates `f` with its unnormalized
(i.e., user-supplied) body.  The latter was created in the prelude
(Section [ref])
by the form `(install-not-normalized f)`.
Notice that no proof by induction was
performed for the 'becomes' theorem (or its local lemma), even though it is
inherently an inductive fact stating the equivalence of
recursive functions `f` and `f$1`.  We are essentially
taking advantage of the trivial induction already performed in the
proof of the lemma being functionally instantiated,
`f$1-is-f$1-copy`.

## Assumptions

The following trivial example shows how assumptions change the `simplify-defun` expansion.

In [34]:
(defun foo (x)
    (declare (xargs :guard (true-listp x)))
    (if (consp x)
        (foo (cdr x))
      x))


The admission of FOO is trivial, using the relation O< (which is known
to be well-founded on the domain recognized by O-P) and the measure
(ACL2-COUNT X).  We observe that the type of FOO is described by the
theorem (NOT (CONSP (FOO X))).  

Computing the guard conjecture for FOO....

The guard conjecture for FOO is trivial to prove, given primitive type
reasoning.  FOO is compliant with Common Lisp.

Summary
Form:  ( DEFUN FOO ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FOO


Under the `:assumption` of the guard, `(true-listp x)`,
the variable `x` in the body is simplified to the constant `nil`, using its context `(not (consp x))`.

In [35]:
(simplify-defun foo :assumptions :guard :show-only t)

(ENCAPSULATE-REPORT-ERRORS
 (APT::SIMPLIFY . FOO)
 NIL (LOGIC)
 (SET-INHIBIT-WARNINGS "theory")
 (SET-IGNORE-OK T)
 (SET-IRRELEVANT-FORMALS-OK T)
 (LOCAL (INSTALL-NOT-NORMALIZED FOO
                                :DEFTHM-NAME 'FOO$NOT-NORMALIZED
                                :ALLP NIL))
 (LOCAL
  (PROGN
   (DEFUN FOO-HYPS (X) (TRUE-LISTP X))
   (MAKE-EVENT
     (LET
      ((APT::THY (MERGE-SORT-LEXORDER
                      (UNION-EQUAL '((:DEFINITION FOO-HYPS))
                                   (CONGRUENCE-THEORY-FN :HERE (W STATE))))))
      (LIST 'DEFCONST
            '*FOO-RUNES*
            (LIST 'QUOTE APT::THY))))
   (ON-FAILURE
    (ORELSE
     (WITH-OUTPUT
      :OFF :ALL
      (DEFUN FOO$1 (X)
       (DECLARE
        (XARGS :GUARD (TRUE-LISTP X)
               :MEASURE (ACL2-COUNT X)
               :VERIFY-GUARDS NIL
               :HINTS
               (("Goal" :INSTRUCTIONS
                        ((:PROVE-TERMINATION FOO *FOO-RUNES*
                                   

This time, the 'becomes' theorem has a hypothesis provided
by the `:assumptions`. The expansion will include:

```lisp
(DEFUN FOO$1 (X)
  (IF (CONSP X) (FOO$1 (CDR X)) NIL))

(DEFTHM FOO-BECOMES-FOO$1
  (IMPLIES (TRUE-LISTP X)
           (EQUAL (FOO X) (FOO$1 X)))
  :HINTS ...)
```

We next discuss some differences in the local events generated when there
are assumptions.  A new local event defines a function for the
assumptions, so that when the assumptions are complicated, disabling
that function can hide complexity from the rewriter.

In [36]:
(DEFUN FOO-HYPS (X) (TRUE-LISTP X))


Since FOO-HYPS is non-recursive, its admission is trivial.  We observe
that the type of FOO-HYPS is described by the theorem 
(OR (EQUAL (FOO-HYPS X) T) (EQUAL (FOO-HYPS X) NIL)).  

Summary
Form:  ( DEFUN FOO-HYPS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FOO-HYPS


In order to prove equivalence of the old and new functions, which
involves a proof by induction at some point, it is necessary to reason
that the assumptions are preserved by recursive calls.  The following
local lemma is generated for that purpose.

In [37]:
(DEFTHM FOO-HYPS-PRESERVED-FOR-FOO
  (IMPLIES (AND (FOO-HYPS X) (CONSP X))
           (FOO-HYPS (CDR X)))
  :HINTS (("Goal" :IN-THEORY (DISABLE* FOO (:E FOO) (:T FOO))
                  :EXPAND ((:FREE (X) (FOO-HYPS X)))
                  :USE (:GUARD-THEOREM FOO)))
  :RULE-CLASSES NIL)

Goal'

Q.E.D.

Summary
Form:  ( DEFTHM FOO-HYPS-PRESERVED-FOR-FOO ...)
Rules: ((:DEFINITION FOO-HYPS)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM))
Hint-events: ((:USE (:GUARD-THEOREM FOO)))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  50
 FOO-HYPS-PRESERVED-FOR-FOO


In many cases the proof of such a lemma will be automatic.
Otherwise, one can first define `foo-hyps` and prove this
lemma before running `simplify-defun`.

There are some tricky wrinkles in the presence of assumptions that we
do not discuss here, in particular how they can affect the use of `copy-def` (discussed above in Section [ref]).  Some relevant discussion may be found in the
``Essay on the Implementation of Simplify-defun'' in the file `simplify-defun.lisp`.

## Implementation Notes

The discussion above explains the expansion from a call of `simplify-defun`.  Here we discuss at a high level how such forms are
generated.  Consider the first example from the introduction, below,
and let us see its single-step macroexpansion.

Consider the first example from the introduction. The single-step macroexpansion of `simplify-defun` shows:

```lisp
(defun f (x)
  (if (zp x) 0 (+ 1 1 (f (+ -1 x)))))

ACL2 !>:trans1 (simplify-defun f)
 (WITH-OUTPUT :GAG-MODE NIL :OFF :ALL :ON ERROR
   (MAKE-EVENT (SIMPLIFY-DEFUN-FN 'F 'NIL 'NIL ':NONE ... STATE)))
```

This use of
[-output](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_WITH-OUTPUT)
prevents output unless there is an error.  The
[-event](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_MAKE-EVENT)
call instructs `simplify-defun-fn` to produce an event
of the form `(progn $E$ $A$ (value-triple '$D$))`,
where $E$ is the expansion, $A$ is a
[](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_TABLE)
event provided to support redundancy for `simplify-defun`, and $D$
is the new definition (which is thus printed to the terminal).  `Simplify-defun-fn` first calls the expander to produce a simplified
body, which it then uses to create $D$ and $E$.  For details see `simplify-defun.lisp` in the supporting materials, which we hope is
accessible to those having a little familiarity with ACL2 system
programming (see for example
[system-utilities](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_SYSTEM-UTILITIES)
and
[programming-with-state](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_PROGRAMMING-WITH-STATE)).
These details help proofs to succeed efficiently, in particular by
generating suitable hints, including small theories.  Another detail
is that if the old definition specifies
[ruler-extenders](http://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index.html?topic=ACL2\_\_\_\_RULER-EXTENDERS)
other than the default, then these are carried over to the new
definition.

# Conclusion

The use of simplification, particularly rewrite rules,
is an old and important idea in program transformation.
`Simplify-defun` realizes this idea in ACL2,
by leveraging the prover's existing
proof procedures, libraries, and environment.
It is one of the transformations of the APT tool suite
for transforming programs and program specifications,
useful for both synthesis and verification.
While `simplify-defun` is appropriate for equivalence-preserving refinements,
other APT transformations are appropriate for other kinds of refinement.
For instance, specifications that allow more than one implementation
(see soft-2015 for an example)
can be refined via APT's narrowing transformation (not discussed here).

We have used `simplify-defun` quite extensively in program derivations,
demonstrating its robustness and utility.
This paper describes not only the general usage of `simplify-defun`,
but also ACL2-specific techniques used to implement it.
It also shows some non-trivial examples that illustrate the tool's utility.

The tool continues to evolve.  Developments since the drafting
of this paper include: delaying guard verification; and expanding
all `LET` expressions, but reconstructing them with `directed-untranslate`.  These enhancements and others will be
incorporated into `simplify-defun` when we add it, soon, to the
community books, located somewhere under `books/kestrel`.